# Project 04 Data Acquisition

In [2]:
import twint
import numpy as np
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

In [3]:
import functools

In [4]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF

In [5]:
# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 400)
pd.options.display.float_format = '{:,.10f}'.format

## Pull in sample data from Twitter using Twint
...showing sample extracts below

In [ ]:
#configuration
config = twint.Config()
config.Search = "@VAVetBenefits"
config.Lang = "en"
config.Limit = 1000
#config.Since = '2020–12–18'
#config.Until = time_until
config.Store_csv = True
config.Output = "custom_out_vba_sample.csv"

#running search
twint.run.Search(config)

In [ ]:
#configuration
config = twint.Config()
config.Search = "@DeptVetAffairs"
config.Lang = "en"
config.Limit = 1000
#config.Since = '2020–12–18'
#config.Until = time_until
config.Store_csv = True
config.Output = "custom_out_VA_sample.csv"

#running search
twint.run.Search(config)

In [ ]:
#configuration
config = twint.Config()
config.Search = "@VeteransHealth"
config.Lang = "en"
config.Limit = 1000
#config.Since = '2020–12–18'
#config.Until = time_until
config.Store_csv = True
config.Output = "custom_out_VHA_sample2.csv"

#running search
vha_sample = twint.run.Search(config)

## Create Combined Dataset 

In [6]:
vba_full = pd.read_csv('/Users/arcarter/Git_Repos/project_04/custom_out_vba.csv')  

In [7]:
vha_full = pd.read_csv('/Users/arcarter/Git_Repos/project_04/custom_out_VHA.csv')

/Users/arcarter/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
va_full = pd.read_csv('/Users/arcarter/Git_Repos/project_04/custom_out_VA.csv')

In [9]:
va_full.shape

(112091, 36)

## Create new columns for VBA vs VHA vs VA Tweets 
(Allowing multiple to be selected for same tweet)

In [10]:
vba_full['VBA Tweet'] = 'Yes'

In [11]:
vba_full_slim = pd.DataFrame(vba_full[["tweet", "id","username","name","date","time","VBA Tweet"]])

In [12]:
vha_full['VHA Tweet'] = 'Yes'

In [13]:
vha_full_slim = pd.DataFrame(vha_full[["tweet", "id","username","name","date","time","VHA Tweet"]])

In [14]:
va_full['VA Tweet'] = 'Yes'

In [15]:
va_full_slim = pd.DataFrame(va_full[["tweet", "id","username","name","date","time","VA Tweet"]])

In [16]:
vba_cols = list(vba_full_slim.columns) 
vha_cols = list(vha_full_slim.columns) 
va_cols = list(va_full_slim.columns) 

In [17]:
no_tweet_vba = vba_cols[:-1]

In [18]:
no_tweet_vha = vha_cols[:-1]

In [19]:
no_tweet_va = va_cols[:-1]

## Merge all versions
...and clean up shared columns

In [20]:
merged_all_slim = pd.merge(left=vba_full_slim, right=va_full_slim, how='outer', left_on=no_tweet_vba, right_on=no_tweet_va)
merged_all_slim.sample(10);

In [42]:
merged_all_slim.shape

(123692, 8)

In [43]:
merged_all_slim[merged_all_slim.username.str.contains("revolutapp")].shape

(34, 8)

In [44]:
merged_all_slim = merged_all_slim[~merged_all_slim.username.str.contains("revolutapp")]

In [45]:
merged_all_slim.shape

(123658, 8)

In [57]:
merged_all_w_vha_slim = pd.merge(left=merged_all_slim, right=vha_full_slim, how='outer', left_on=no_tweet_vba, right_on=no_tweet_vha)

In [58]:
merged_all_w_vha_slim.head(5)

,tweet,id,username,name,date,time,VBA Tweet,VA Tweet,VHA Tweet
0,@RevolutApp can i have a response,1361860276115636224,ugurpacaci1,Ugur Pacaci,2021-02-16,21:09:26,Yes,NaN,NaN
1,@RevolutApp hi.. Can someone who is in the UK as a tourist sign up for Revolut virtual card?,1361856147553148931,dr_kazi_uddin,Dr. Kazi Uddin,2021-02-16,20:53:02,Yes,NaN,NaN
2,@RevolutApp Yess,1361851928402145281,ayakaaa89711346,Ayakaaa,2021-02-16,20:36:16,Yes,NaN,NaN
3,@RevolutApp Please help I can’t sign in to my app!!!,1361851688357920768,ayakaaa89711346,Ayakaaa,2021-02-16,20:35:19,Yes,NaN,NaN
4,@RevolutApp This is what you say every time and nothing gets sorted,1361842815500759043,partiean,✨,2021-02-16,20:00:03,Yes,NaN,NaN


In [59]:
merged_all_w_vha_slim.shape

(219778, 9)

In [60]:
merged_all_w_vha_slim = merged_all_w_vha_slim[~merged_all_w_vha_slim.tweet.str.contains("RevolutApp")]
merged_all_w_vha_slim = merged_all_w_vha_slim[~merged_all_w_vha_slim.username.str.contains("revolutapp")]

In [61]:
merged_all_w_vha_slim.shape

(219712, 9)

In [62]:
merged_all_w_vha_slim.loc[merged_all_w_vha_slim['VBA Tweet'] == 'Yes', 'VBA Tweet'] = 1
merged_all_w_vha_slim.loc[merged_all_w_vha_slim['VA Tweet'] == 'Yes', 'VA Tweet'] = 1
merged_all_w_vha_slim.loc[merged_all_w_vha_slim['VHA Tweet'] == 'Yes', 'VHA Tweet'] = 1

In [63]:
merged_all_w_vha_slim['VBA Tweet'] = merged_all_w_vha_slim['VBA Tweet'].fillna(0)
merged_all_w_vha_slim['VA Tweet'] = merged_all_w_vha_slim['VA Tweet'].fillna(0)
merged_all_w_vha_slim['VHA Tweet'] = merged_all_w_vha_slim['VHA Tweet'].fillna(0)

In [64]:
merged_all_w_vha_slim.head(5)

,tweet,id,username,name,date,time,VBA Tweet,VA Tweet,VHA Tweet
66,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
67,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
68,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
69,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
70,"RT @VAVetBenefits : (2/2) In 2014, Howard became the first female four-star Admiral &amp; the first to command operational forces, making her the highest ranking woman &amp; African-American in Navy history. After a remarkable career, Adm. Howard retired in 2017. Thank you for your …",1361854662488555525,lccc_sva,LCCC SVA,2021-02-16,20:47:08,1,0,0


In [73]:
merged_all_w_vha_slim[merged_all_w_vha_slim['username'].str.contains("247fubar")].head(100)

,tweet,id,username,name,date,time,VBA Tweet,VA Tweet,VHA Tweet
1768,@ConcernedVets @DeptVetAffairs @HouseVetAffairs @VAVetBenefits @SpecialReport @RepCharlieCrist @SenRickScott @marcorubio @TB_Times @realannapaulina Pls clarify those Florida news reports on Vets’ income and Covid vaccine access. Thank you.,1352587376791588868,247fubar,Gary,2021-01-22,07:02:15,1,1,0
1769,@ConcernedVets @DeptVetAffairs @HouseVetAffairs @VAVetBenefits @SpecialReport @RepCharlieCrist @SenRickScott @marcorubio @TB_Times @realannapaulina Pls clarify those Florida news reports on Vets’ income and Covid vaccine access. Thank you.,1352587376791588868,247fubar,Gary,2021-01-22,07:02:15,1,1,0
1770,@ConcernedVets @DeptVetAffairs @HouseVetAffairs @VAVetBenefits @SpecialReport @RepCharlieCrist @SenRickScott @marcorubio @TB_Times @realannapaulina Pls clarify those Florida news reports on Vets’ income and Covid vaccine access. Thank you.,1352587376791588868,247fubar,Gary,2021-01-22,07:02:15,1,1,0
1771,@ConcernedVets @DeptVetAffairs @HouseVetAffairs @VAVetBenefits @SpecialReport @RepCharlieCrist @SenRickScott @marcorubio @TB_Times @realannapaulina Pls clarify those Florida news reports on Vets’ income and Covid vaccine access. Thank you.,1352587376791588868,247fubar,Gary,2021-01-22,07:02:15,1,1,0


## Send version to CSV
...and Pickle

In [65]:
merged_all_w_vha_slim.to_csv(index=False,path_or_buf='/Users/arcarter/Git_Repos/project_04/merged_all_w_vha_slim.csv')

In [66]:
merged_all_w_vha_slim.shape

(219712, 9)

In [67]:
import pickle

In [68]:
pickle.dump(merged_all_w_vha_slim, open("merged_all_w_vha_slim.p", "wb" ) )

In [69]:
merged_all_w_vha_slim_pickle = pickle.load( open( "merged_all_w_vha_slim.p", "rb" ) )

In [70]:
merged_all_w_vha_slim.head()

,tweet,id,username,name,date,time,VBA Tweet,VA Tweet,VHA Tweet
66,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
67,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
68,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
69,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
70,"RT @VAVetBenefits : (2/2) In 2014, Howard became the first female four-star Admiral &amp; the first to command operational forces, making her the highest ranking woman &amp; African-American in Navy history. After a remarkable career, Adm. Howard retired in 2017. Thank you for your …",1361854662488555525,lccc_sva,LCCC SVA,2021-02-16,20:47:08,1,0,0
